### This script will load some actual trajectory data and see how well the model fits
First import the maths and plotting packages we used before

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import csv

#### We need to load some trajectory data
These are real trajectories from a table tennis ball.

There are three columns; [x, y, velocity].

For now, we're going to load trajectory 75 and separate out the parameters (note different parameter names)

In [ ]:
trajectory=75
data = np.genfromtxt('trajectories/trajectory'+str(trajectory)+'.txt',delimiter=',');
sx = data[:,0];
sy = data[:,1];
vx = data[:,2];
vy = data[:,3];

#### Let's plot the trajectory
Each one starts at zero on the x-axis and ends when the y value reaches zero (when the ball hit the table). The launch height and angle are variable

In [ ]:
...
...
...
...

#### We also have the velocities already calculated. Let's plot these
We'll first need to define a time variable. We know the sampling rate was 250Hz

In [ ]:
sampleRate=250
t=np.arange(0, len(sx)/sampleRate,1/sampleRate)

#### Now plot the horizontal and vertical velocity
As you can see the horizontal velocity isn't constant. What causes this and what effect will it have?

In [ ]:
...
...
...
...

#### We now need to define the starting parameters for the model to work

In [ ]:
v0vert=vy[0]
v0horz=vx[0]
h0=sy[0]
theta0=np.degrees(np.arctan(vy[0]/vx[0]))

#### We're going to define a custom function called 'model_trajectory' that will do all the calculations in one go.
Once its been defined, a function can then be called anytime in the script. You've already used several built in ones such as 'np.degrees' and 'np.arange'

In [ ]:
def model_trajectory(v0vert,v0horz,h0,theta0,t):
        
    svert = h0 + v0vert*t-0.5*9.81*np.square(t)
    shorz = v0horz*t

    return svert,shorz

#### We can now call this function and model the trajectory based on the starting parameters

In [ ]:
svert,shorz=model_trajectory(v0vert,v0horz,h0,theta0,t)

#### Let's see how they compare

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(sx,sy,label='Data');
ax.plot(shorz,svert,label='Model');
ax.axis('scaled')
ax.set_xlabel('x (m)');
ax.set_ylabel('y (m)');
ax.legend();

#### It's not terrible but the model overpredicts the range
What if we take into account air resistance using the new function below? This gets a bit more complicated because now the velocity changes constantly through time so we have to keep updating it using a for loop.

We'll also make this model do all the initial formatting of the data so it can be quickly run on other datasets

In [ ]:
def model_trajectory_air_resistance(data,sampleRate,S,m,CD0):

    # format data
    sx = data[:,0];
    sy = data[:,1];
    vx = data[:,2];
    vy = data[:,3];
    
    v0vert=vy[0]
    v0horz=vx[0]
    h0=sy[0]
    theta0=np.degrees(np.arctan(vy[0]/vx[0]))
    
    # get time series
    t=np.arange(0, len(sx)/sampleRate,1/sampleRate)

    # fixed parameters
    rho = 1.225; # air density
    g = 9.81 # acceleration due to gravity
    
    vy = v0vert
    vx = v0horz
    theta = theta0
    
    shorz=np.array([0])
    svert=np.array([h0])
        
    t_step = 1/sampleRate # time step
    
    cont=1
    
    while cont==1:
        
        vtot=np.hypot(vx,vy) # total velocity

        ax=-0.5*1/m*rho*vtot*S*(vx*CD0); # horizontal acceleration
        ay=-0.5*1/m*rho*vtot*S*(vy*CD0)-g; # vertical acceleration
    
        # update variables
        vx=vx + ax*t_step; # horizontal velocity
        vy=vy + ay*t_step; # vertical velocity
        
        shorznew=shorz[-1] + vx*t_step + 0.5*ax*np.square(t_step);
        svertnew=svert[-1] + vy*t_step + 0.5*ay*np.square(t_step);
        
        if svertnew<0:
            cont=0
        else:
            shorz = np.append(shorz,shorznew)
            svert = np.append(svert,svertnew)
     
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(sx,sy,label='Data');
    ax.plot(shorz,svert,label='Model');
    ax.axis('scaled')
    ax.set_xlabel('x (m)');
    ax.set_ylabel('y (m)');
    ax.legend();

    return svert,shorz

#### We have a few new parameters to define
S = projected surface area of the ball (m)

m = mass of the ball (kg)

CD0 = drag coefficient

In [ ]:
S = 0.00125 # fixed, based on a standard 40mm diameter ball
m = 0.0027 # fixed, based on a standard 2.7g ball
CD0 = 0.2 # from literature, but only an estimate so could be varied

#### Run the function and see if it improves things
Try modifying CD0 and see if that improves things

In [ ]:
svert,shorz=model_trajectory_air_resistance(data,sampleRate,S,m,CD0)

#### You can get a pretty good result with the right value of CD0
But what if we try a different sequence?

Let's load up a different dataset and run the function. You can go up trajectory = 82

In [ ]:
trajectory=...
data = np.genfromtxt('trajectories/trajectory'+str(trajectory)+'.txt',delimiter=',');
svert,shorz=model_trajectory_air_resistance(data,sampleRate,S,m,CD0)

#### Some are ok, but there are also a lot that haven't worked well. What are we missing?
Let's load up some data on the ball spin rates

#### We can plot a histogram of them and indicate the 75th trajectory
Topspin is negative, backspin is positive

The 75th sequence had almost no spin, which is why it worked quite well before

In [ ]:
spinrates = np.genfromtxt('spinrates.txt',delimiter=',');
fig, ax=plt.subplots(1)
ax.hist(spinrates)
ax.axvline(spinrates[74],color = 'r') # zero-based indexing so have to use -1
ax.plot()
ax.set(xlabel='Spinrate (rev/s)');

#### So we can make a final model that incorporates spin.
This will affect both lift and drag

In [ ]:
def model_trajectory_full_model(data,sampleRate,S,m,CD0,CLa,omega):

    # format data
    sx = data[:,0];
    sy = data[:,1];
    vx = data[:,2];
    vy = data[:,3];
    
    v0vert=vy[0]
    v0horz=vx[0]
    h0=sy[0]
    theta0=np.degrees(np.arctan(vy[0]/vx[0]))
    
    # get time series
    t=np.arange(0, len(sx)/sampleRate,1/sampleRate)

    # fixed parameters
    rho = 1.225; # air density
    g = 9.81 # acceleration due to gravity
    
    vy = v0vert
    vx = v0horz
    theta = theta0
    
    shorz=np.array([0])
    svert=np.array([h0])
        
    t_step = 1/sampleRate # time step
    
    # lift and drag coefficients taking into account spin
    CL=(-CLa*np.exp(-0.25*np.absolute(omega))+CLa)*np.sign(omega);
    CD=CD0+0.005*np.absolute(omega);

    cont=1
    
    while cont==1:
        
        vtot=np.hypot(vx,vy) # total velocity

        ax=-0.5*1/m*rho*vtot*S*(vx*CD+vy*CL); # horizontal acceleration
        ay=-0.5*1/m*rho*vtot*S*(vy*CD-vx*CL)-g; # vertical acceleration
    
        # update variables
        vx=vx + ax*t_step; # horizontal velocity
        vy=vy + ay*t_step; # vertical velocity
        
        shorznew=shorz[-1] + vx*t_step + 0.5*ax*np.square(t_step);
        svertnew=svert[-1] + vy*t_step + 0.5*ay*np.square(t_step);
        
        if svertnew<0:
            cont=0
        else:
            shorz = np.append(shorz,shorznew)
            svert = np.append(svert,svertnew)
     
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(sx,sy,label='Data');
    ax.plot(shorz,svert,label='Model');
    ax.axis('scaled')
    ax.set_xlabel('x (m)');
    ax.set_ylabel('y (m)');
    ax.legend();

    return svert,shorz

#### Let's try with some trajectories and run the final model
Try changing the values of CD0 and CLa to see their effect. Can you find a combination that seems optimal?

In [ ]:
trajectory=80
data = np.genfromtxt('trajectories/trajectory'+str(trajectory)+'.txt',delimiter=',');
omega=spinrates[trajectory-1]

CD0=...
CLa=...

svert,shorz=model_trajectory_full_model(data,sampleRate,S,m,CD0,CLa,omega)